In [ ]:
import pandas as pd
import time
import re 
import seaborn as sns
import numpy as np

from tqdm._tqdm_notebook import tqdm_notebook
import numpy as np
tqdm_notebook.pandas()

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [ ]:
def vader_by_line(lyrics):
    split = lyrics.splitlines()
    lines = list(filter(None, split))
    scores = []
    for line in lines:
        score = analyser.polarity_scores(line)
        scores.append(score['compound'])
    scores = list(filter(lambda x: x != 0.0, scores))
    if len(scores) == 0:
        return np.nan
    return np.mean(scores)

In [ ]:
song_info = pd.read_csv('../data/lyrics/song_info.txt')
song_info = song_info[song_info['lang'] == 'en']
song_info = song_info[['song_id', 'title', 'artist', 'lyrics', 'audio_features']]
song_info.head()

In [ ]:
reg = "[\(\[].*?[\)\]]"
print('Original length:', len(song_info))
song_info['lyrics'] = song_info['lyrics'].str.replace(reg, '')
song_info = song_info[song_info['lyrics'] != '']
print('Cleaned length:', len(song_info))

In [ ]:
song_info.head()

In [ ]:
song_info['vader_score'] = song_info['lyrics'].progress_apply(lambda x: vader_by_line(x))

In [ ]:
song_info[song_info['vader_score'].isnull()]

In [ ]:
vader_df = song_info[~song_info['vader_score'].isnull()].copy()
vader_df[['song_id', 'vader_score']].to_csv(path_or_buf='../data/vader_scores.txt', index=False,)

In [ ]:
vader_df['vader_score'].describe()

In [ ]:
sns.distplot(vader_df['vader_score'])